# Preprocesamiento de datos (Tweets)

En este notebook se realiza el preprocesamiento de los datos que contienen los tweets para el entrenamiento de los modelos de aprendizaje automático extraidos de HuggingFace. Los datos han dido extraídos de la plataforma de Kaggle, en el siguiente [enlace](https://www.kaggle.com/datasets/alishafaghi/hashtag-tesla-tweets). El dataset contiene más de 150.000 tweets etiquetados con #tesla.

El objetivo es obtener un dataset con una única columna de tweets en inglés, para su posterior etiquetado por parte del alumno. El etiquetado de datos será discutido en la memoria, y realizado en los scripts de la carpeta `labelling`.

In [1]:
import pandas as pd
import yfinance as yf
from langdetect import detect
import preprocessor as p
import string
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER)

KeyboardInterrupt: 

## Preprocesamiento del dataset de tweets

### Overview

En esta sección se realiza la carga de datos y análisis de alto nivel del datafrfame

In [ ]:
df_tweets = pd.read_csv('../../data/raw/tesla_tweets.csv')
df_tweets.sample(5)

,Date & Time,Profile Picture Link,Twitter ID,Tweet Text,Tweet Link
51865,"June 25, 2022 at 01:06AM",http://pbs.twimg.com/profile_images/346791608...,@MapleDee,RT @YRP: A LITTLE MORE MONEY THAN SENSE: A $19...,https://twitter.com/MapleDee/status/1540433784...
73629,"July 25, 2022 at 08:20AM",http://pbs.twimg.com/profile_images/139665024...,@RahulNe11595917,RT @ani_digital: Elon Musk had an affair with ...,https://twitter.com/RahulNe11595917/status/155...
100114,"August 31, 2022 at 08:17PM",http://pbs.twimg.com/profile_images/153866697...,@franksleash08,#USA #RUS #CN #JP #SA #KOR #PAR #UK\n#DXB #DEU...,https://twitter.com/franksleash08/status/15650...
12521,"April 30, 2022 at 02:20PM",http://pbs.twimg.com/profile_images/151448831...,@InterMinabileXa,RT @Mentegatto1: @InterMinabileXa @elonmusk @I...,https://twitter.com/InterMinabileXa/status/152...
67343,"July 16, 2022 at 02:17PM",http://pbs.twimg.com/profile_images/412169901...,@ProVega350,It's time for leadership in #America. Making t...,https://twitter.com/ProVega350/status/15482428...


Como lo único que nos interesa es la columna de texto y su id, se descartan las demás columnas y se establece el tweet de cada post como índice del dataframe.

In [ ]:

selected_columns = ['Tweet Link','Date & Time','Tweet Text']
df_tweets = df_tweets[selected_columns]
df_tweets.rename(columns={'Tweet Text':'Text', 'Tweet Link' : 'Link', 'Date & Time' : 'Date'}, inplace=True)
df_tweets.sample(5)

,Link,Date,Text
10725,https://twitter.com/crandypants/status/1518360...,"April 25, 2022 at 03:14AM",RT @Gangpeh76: @crandypants @elonmusk @WholeMa...
18124,https://twitter.com/wendy53581/status/15232075...,"May 08, 2022 at 12:16PM",RT @Wanli_Xiong: Hug Mother’s Day! #Tesla @elo...
21756,https://twitter.com/serkankaradag80/status/152...,"May 14, 2022 at 12:08AM","Twitter ne yapsın,coinlerdeki sert düşüşte fay..."
14316,https://twitter.com/EnriqueSimo28/status/15212...,"May 03, 2022 at 04:10AM",RT @GerberKawasaki: Hey @elonmusk what if we w...
108329,https://twitter.com/Gambiste1/status/156915574...,"September 12, 2022 at 07:17AM",RT @KentaroYamada5: #Tesla would likely also b...


In [ ]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152000 entries, 0 to 151999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Link    152000 non-null  object
 1   Date    152000 non-null  object
 2   Text    152000 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB


Comprobamos que la columna de fecha no está en formato datetime, por lo que se convierte a dicho formato. 

In [ ]:
df_tweets['Date'] = pd.to_datetime(df_tweets['Date'], format='mixed') # dejamos que pandas infiera el formato de fecha con el formato mixed
print(df_tweets.info())
df_tweets.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152000 entries, 0 to 151999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Link    152000 non-null  object        
 1   Date    152000 non-null  datetime64[ns]
 2   Text    152000 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 3.5+ MB
None


,Link,Date,Text
0,https://twitter.com/Jessica1988kk/status/15131...,2022-04-10 19:44:00,"RT @invest_answers: Crypto news, #Bitcoin Whal..."
1,https://twitter.com/JotaGe2014/status/15131737...,2022-04-10 19:45:00,#Tesla tiene récord de autos vendidos. Es impr...
2,https://twitter.com/MmeCallas/status/151317374...,2022-04-10 19:45:00,RT @CottonCodes: 🐒 #love in my #MariaCallas I ...
3,https://twitter.com/BotSecx/status/15131737626...,2022-04-10 19:45:00,RT @CottonCodes: 🐒 #love in my #MariaCallas I ...
4,https://twitter.com/agseh/status/1513173864829...,2022-04-10 19:45:00,RT @RupiReportero_: 🙆‍♂️🚘 Al que le robaron la...


### Limpieza de datos

Se comprueba si existen valores nulos o atípicos, además de comprobar si existen tweets duplicados.


In [ ]:
df_tweets.isnull().sum()

Link    0
Date    0
Text    0
dtype: int64

Al ser la fecha de publicación del tweet y el propio tweet en cuestion variables de tipos distintos, se hace su análisis por separado.

En la descripción del dataset en la página de Kaggle se indica que los tweets han sido extraídos entre Abril de 2022 y Noviembre de 2022. Por lo que se comprueba que no existen tweets con fecha anterior al 1 de enero de 2020. Comprobamos que no existen tweets con fechas posteriores ni anteriorers a las indicadas.

In [ ]:
# Análisis de valores atípicos en la columna de fecha

(df_tweets['Date'].min(),
df_tweets['Date'].max())

(Timestamp('2022-04-10 19:44:00'), Timestamp('2022-11-12 15:11:00'))

### Preparación de la columna de text

En esta sección se realiza la limpieza de la columna de texto.Se eliminan los caracteres especiales, las menciones, los enlaces y los emojis utilizando la librería [tweet preprocessor](https://pypi.org/project/tweet-preprocessor/). Se convierte todo el texto a minúsculas, y seleccionamos sólo los tweets en inglés.	

Sería interesante poder elegir otros idiomas aparte del inglés para el etiquetado de datos, pero por simplicidad se ha decidido utilizar sólo el inglés, ya que es el idioma más utilizado en la plataforma de Twitter. Sería posible utilizar modelos preentrenados para la clasificación de sentimientos en diferentes idiomas, pero se ha decidido utilizar sólo el inglés para simplificar el proceso de etiquetado.

In [ ]:
#dataframe vacío para almacenar los tweets que son en inglés
i = 0
df_aux = pd.DataFrame(columns=['Link','Date','Text'])
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER)
start_time = time.time()

for index, row in df_tweets.iterrows():
    cleaned_text = p.clean(row['Text'])  # Limpia el texto y guarda el resultado

    # Esta celda tarda mucho, por lo que si se desea saber el progreso se puede imprimir el porcentaje de tweets procesados
    if i % (len(df_tweets.index)/10) == 0:
        print('Progreso:',i,'/',len(df_tweets.index),'=',i/len(df_tweets.index)*100,'%')
        print('Tiempo transcurrido:',time.time()-start_time,'segundos')
    i+=1
    
    try:
        if detect(cleaned_text) == 'en': # Si el texto está en inglés, lo guardamos en el dataframe auxiliar
            df_aux.loc[len(df_aux.index)] = [row['Link'], row['Date'], cleaned_text]
    except:
        pass


Progreso: 0 / 152000 = 0.0 %
Tiempo transcurrido: 0.5475349426269531 segundos


In [ ]:
def eliminar_puntuacion(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def eliminar_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

stop_words = set(stopwords.words('english'))

df_aux['Text'] = df_aux['Text'].apply(eliminar_puntuacion)
df_aux['Text'] = df_aux['Text'].apply(eliminar_stopwords)

print(len(df_aux.index))
df_aux.head(5)

124676


,Link,Date,Text
0,https://twitter.com/Jessica1988kk/status/15131...,2022-04-10 19:44:00,Crypto news Bitcoin Whales flying motorbikes w...
1,https://twitter.com/MmeCallas/status/151317374...,2022-04-10 19:45:00,love MariaCallas I KNOW Y art HOLDonLINE music...
2,https://twitter.com/BotSecx/status/15131737626...,2022-04-10 19:45:00,love MariaCallas I KNOW Y art HOLDonLINE music...
3,https://twitter.com/ElTendies/status/151317393...,2022-04-10 19:45:00,Tesla A Trillion Dollar Company Worlds Largest...
4,https://twitter.com/LauraCory2013/status/15131...,2022-04-10 19:45:00,chargingstations area I dont Tesla Powerwall a...


Como también fue notado en la caracterización de los datos, existen días que los mercados no están abiertos y por tanto no hay observaciones de las acciones. La estrategia de entrenamiento de los modelos de aprendizaje automático será la de predecir el precio de las acciones al día siguiente, por lo que se decide eliminar los tweets de los días en los que no hay observaciones de las acciones.

In [ ]:
df_stocks = pd.read_csv('../../data/raw/tesla_stocks.csv')
df_stocks['Date'] = pd.to_datetime(df_stocks['Date'], format='mixed')

df_copy = df_aux.copy()


mask = df_copy['Date'].isin(df_stocks['Date'])
df_copy = df_copy[mask]

print(len(df_copy.index))


9


In [ ]:
df_aux.to_csv('../../data/clean/tesla_tweets_unlabelled.csv', index=False)